### Pruebas para detectar pfam promedio por target en chembl

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from importlib import reload

In [2]:
import pandas as pd
import numpy as np
import collections

### Database connexion

In [3]:
# Personal login details in correct format for sqlalchemy

pepe ="sqlite:////home/fleer/Desktop/Tesis/Chembldb/chembl_27/chembl_27_sqlite/chembl_27/chembl_27_sqlite/chembl_27.db"
engine = create_engine(pepe)
CHEMBL_VERSION = 27

In [8]:
sql = '''
SELECT td.chembl_id as target_chemblid, 
       a2.pchembl_value, a2.activity_comment, 
       md.chembl_id as compound_chemblid, 
       source_domain_id, 
       dm.mec_id
FROM drug_mechanism dm
    JOIN binding_sites bs on bs.tid = dm.tid
    JOIN target_dictionary td ON td.tid = bs.tid 
    JOIN site_components sc ON sc.site_id =bs.site_id
    JOIN domains d2 ON d2.domain_id = sc.domain_id
    JOIN activities a2 ON dm.molregno = a2.molregno
    JOIN molecule_dictionary md ON md.molregno = dm.molregno
    JOIN compound_properties cp ON cp.molregno = md.molregno
    JOIN compound_records cr ON cr.molregno = cp.molregno
WHERE a2.src_id = 15
    AND a2.standard_type = 'IC50'
    AND dm.mec_id IS NOT NULL 
    AND a2.activity_comment LIKE 'Active'
    AND cp.PSA IS NOT NULL
UNION 
SELECT td.chembl_id as target_chemblid, 
       a2.pchembl_value, a2.activity_comment, 
       md.chembl_id as compound_chemblid, 
       source_domain_id, 
       dm.mec_id
FROM drug_mechanism dm
    JOIN binding_sites bs on bs.tid = dm.tid
    JOIN target_dictionary td ON td.tid = bs.tid 
    JOIN site_components sc ON sc.site_id =bs.site_id
    JOIN domains d2 ON d2.domain_id = sc.domain_id
    JOIN activities a2 ON dm.molregno = a2.molregno
    JOIN molecule_dictionary md ON md.molregno = dm.molregno
    JOIN compound_properties cp ON cp.molregno = md.molregno
    JOIN compound_records cr ON cr.molregno = cp.molregno
WHERE a2.src_id = 15
    AND a2.standard_type = 'IC50'
    AND dm.mec_id IS NOT NULL 
    AND a2.pchembl_value >= 6.0
    AND cp.PSA IS NOT NULL;
  '''
df = pd.read_sql(sql, engine)

In [9]:
df.shape

(556, 6)

In [10]:
df.head()

,target_chemblid,pchembl_value,activity_comment,compound_chemblid,source_domain_id,mec_id
0,CHEMBL1798,6.27,None,CHEMBL603,PF00001,71
1,CHEMBL1798,6.45,None,CHEMBL603,PF00001,71
2,CHEMBL1798,8.76,None,CHEMBL603,PF00001,71
3,CHEMBL1808,6.89,None,CHEMBL1560,PF01401,114
4,CHEMBL1856,6.00,None,CHEMBL710,PF02544,266


In [11]:
df=df.drop_duplicates(subset= ['target_chemblid', 'source_domain_id', 'compound_chemblid'])
df.shape

(120, 6)

In [13]:
df.to_csv(r'/home/fleer/Desktop/pfamid_mec_df_nodup.csv') #paso intermedio

In [14]:
grouped_df = df.groupby("target_chemblid")
grouped_pfam = grouped_df["source_domain_id"].apply(list)


In [15]:
grouped_df = df.groupby("target_chemblid")
grouped_pfam = grouped_df["source_domain_id"].apply(list)
grouped_pfam = grouped_pfam.reset_index()
grouped_pfam['Domain_key']=['_'.join(sorted(set(x))) for x in grouped_pfam.source_domain_id]
grouped_pfam.head()



,target_chemblid,source_domain_id,Domain_key
0,CHEMBL1798,[PF00001],PF00001
1,CHEMBL1808,[PF01401],PF01401
2,CHEMBL1856,[PF02544],PF02544
3,CHEMBL1871,"[PF00104, PF00104, PF00104, PF00104, PF00104, ...",PF00104
4,CHEMBL1875,[PF00001],PF00001


In [16]:
grouped_compound = grouped_df["compound_chemblid"].apply(list)
grouped_compound = grouped_compound.reset_index()
#grouped_compound.head()
result = pd.merge(grouped_pfam, grouped_compound, how='left', on=['target_chemblid'])
result.head()

,target_chemblid,source_domain_id,Domain_key,compound_chemblid
0,CHEMBL1798,[PF00001],PF00001,[CHEMBL603]
1,CHEMBL1808,[PF01401],PF01401,[CHEMBL1560]
2,CHEMBL1856,[PF02544],PF02544,[CHEMBL710]
3,CHEMBL1871,"[PF00104, PF00104, PF00104, PF00104, PF00104, ...",PF00104,"[CHEMBL1479, CHEMBL1200623, CHEMBL1395, CHEMBL..."
4,CHEMBL1875,[PF00001],PF00001,[CHEMBL1729]


In [18]:
result.to_csv ( r'/home/fleer/Desktop/pfamid_chembl27_mech.csv')

### Analisis

In [8]:
sum_df = df.groupby("target_chemblid").count()
sum_df.head()

,source_domain_id,compound_chemblid,pchembl_value,activity_comment
target_chemblid,,,,
CHEMBL1075021,132,132,132,0
CHEMBL1075022,88,88,88,0
CHEMBL1075023,3,3,3,0
CHEMBL1075024,17,17,17,0
CHEMBL1075025,16,16,16,0


In [9]:
sum_df.shape

(5729, 4)

In [ ]:
ax = sum_df.hist(column= 'pchembl_value', bins=200, figsize=(18, 18)) #grafico de numero de pfamid por chembl id

In [ ]:
"source_domain_id", 